In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from google.colab import drive



In [ ]:
# Mount Google Drive (if dataset is stored there)
drive.mount('/content/drive')

# Paths to your dataset (adjust according to your dataset path in Google Drive)
train_dir = '/content/drive/MyDrive/train'
test_dir = '/content/drive/MyDrive/test'



Mounted at /content/drive


In [ ]:
# Image data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Loading the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG19 expects 224x224 input images
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (covid vs normal)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2754 images belonging to 2 classes.
Found 1288 images belonging to 2 classes.


In [ ]:

# Loading the pre-trained VGG19 model without the top classification layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Adding custom classification layers on top of VGG19
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on dataset size
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('/content/vgg19_covid19_classifier.h5')

# Optionally: evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')
